In [1]:
import torch
import torch.nn as nn
import torch.utils.data as data_utils
from torch.utils.data import DataLoader, TensorDataset
import time

class NeuralNetwork(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super().__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        return x

#Load data tensors
input_data = torch.load(r'C:\Users\Diana22\Documents\Andrew\Houston Geophysics\SimPEG\simpeg\SimPEG\electromagnetics\natural_source\input_data.pt')
target_data = torch.load(r'C:\Users\Diana22\Documents\Andrew\Houston Geophysics\SimPEG\simpeg\SimPEG\electromagnetics\natural_source\target_data.pt')
input_tensor = torch.tensor(input_data, dtype=torch.float32)
target_tensor = torch.tensor(target_data, dtype=torch.float32)
# Check for NaN values in the input and target tensors
if torch.isnan(input_tensor).any() or torch.isnan(target_tensor).any():
    print("Input or target data contains NaN values.")

#Split data into training and testing sets
split_ratio = 0.8
split = int(len(input_tensor) * split_ratio)
train_input = input_tensor[:split]
train_target = target_tensor[:split]
test_input = input_tensor[split:]
test_target = target_tensor[split:]

#Create data loaders for training and testing
train_dataset = TensorDataset(train_input, train_target)
test_dataset = TensorDataset(test_input, test_target)
batch_size = 64
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size)

#Build model architecture
input_size = 56  #Input dimension
hidden_size = 128  #Number of neurons in the hidden layer
output_size = 51  #Output dimension
model = NeuralNetwork(input_size, hidden_size, output_size)
criterion = nn.MSELoss()  # Mean Squared Error
optimizer = torch.optim.SGD(model.parameters(), lr=0.0001)  # Stochastic Gradient Descent

#Run Model
start_time = time.time()
num_epochs = 100
for epoch in range(num_epochs):
    for inputs, targets in train_loader:
        #Forward pass
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        
        #Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    #if (epoch + 1) % 10 == 0:
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')
print("--- %s seconds ---" % (time.time() - start_time))    

#Evaluate model on test dataset
model.eval()
total_loss = 0.0
with torch.no_grad():
    for inputs, targets in test_loader:
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        total_loss += loss.item()
mean_loss = total_loss / len(test_loader)
print(f"Mean squared error on test set: {mean_loss:.4f}")

Epoch [1/100], Loss: 1.7091
Epoch [2/100], Loss: 1.7064
Epoch [3/100], Loss: 1.0668
Epoch [4/100], Loss: 1.1784
Epoch [5/100], Loss: 1.7338
Epoch [6/100], Loss: 1.1413
Epoch [7/100], Loss: 0.9996
Epoch [8/100], Loss: 1.1315
Epoch [9/100], Loss: 1.4301
Epoch [10/100], Loss: 0.8046
Epoch [11/100], Loss: 1.1042
Epoch [12/100], Loss: 1.1280
Epoch [13/100], Loss: 1.0796
Epoch [14/100], Loss: 1.3798
Epoch [15/100], Loss: 1.3294
Epoch [16/100], Loss: 0.9719
Epoch [17/100], Loss: 1.2647
Epoch [18/100], Loss: 0.8902
Epoch [19/100], Loss: 1.2640
Epoch [20/100], Loss: 1.0560
Epoch [21/100], Loss: 1.0128
Epoch [22/100], Loss: 1.0260
Epoch [23/100], Loss: 0.7831
Epoch [24/100], Loss: 0.9826
Epoch [25/100], Loss: 1.4265
Epoch [26/100], Loss: 1.3101
Epoch [27/100], Loss: 1.1713
Epoch [28/100], Loss: 1.2767
Epoch [29/100], Loss: 1.4849
Epoch [30/100], Loss: 1.0849
Epoch [31/100], Loss: 1.1831
Epoch [32/100], Loss: 1.0376
Epoch [33/100], Loss: 1.3667
Epoch [34/100], Loss: 1.2993
Epoch [35/100], Loss: 1